<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/bi_lstm_crf_500epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==1.14.0
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.14.0
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install seqeval==0.0.5

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 3.2MB 48.2MB/s 
     |████████████████████████████████| 491kB 53.3MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 317kB 17.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 377.0MB 45kB/s 
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-h7tmwyn8
  Running command git clone -q 

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive/')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Mounted at /content/drive/


In [ ]:
import ast
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Train.csv') #Set Dir
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Test.csv') #Set Dir

In [ ]:
train_df['words'] = train_df['words'].apply(ast.literal_eval)
train_df['tags'] = train_df['tags'].apply(ast.literal_eval)
test_df['words'] = test_df['words'].apply(ast.literal_eval)
test_df['tags'] = test_df['tags'].apply(ast.literal_eval)

train_words = train_df.words
train_tags = train_df.tags

test_words = test_df.words
test_tags = test_df.tags

In [ ]:
words = list(train_words) + list(test_words)
tags = list(train_tags) + list(test_tags)

In [ ]:
pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 139kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
model = gensim.models.Word2Vec(words, size = 300, min_count = 7)

In [ ]:
#model.wv.get_vector('Record')

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
X_train, X_test, y_train, y_test = train_words, test_words, train_tags, test_tags

In [ ]:
print(type(X_train))

<class 'pandas.core.series.Series'>


In [ ]:
print(X_train)

0       [Record, date:, 2083-10-18, CARDIOLOGY, WETZEL...
1       [25, years., He, is, not, married,, no, childr...
2       [tracing, is, Unchanged, Selected, recent, lab...
3       [Record, date:, 2081-04-13, CARDIOLOGY, ALGIER...
4       [history:, Unchanged, from, previous, visits.,...
                              ...                        
2374    [is, soft, and, non-tender,, no, skin, rash., ...
2375    [due, to, cardiac, stents., 2., Upon, review, ...
2376    [Record, date:, 2074-02-22, Gregg,, Lloyd, 2-2...
2377    [cut, back, because, his, wife, has, a, proble...
2378    [I, think, he, found, this, sobering., 3., Gen...
Name: words, Length: 2379, dtype: object


In [ ]:
# Tokenize
max_words = 70000
src_tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')
src_tokenizer.fit_on_texts(words)

tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(tags)

In [ ]:
vocab_size = max_words
tag_size = len(tar_tokenizer.word_index) + 1
print('단어 집합의 크기 : {}'.format(vocab_size))
print('개체명 태깅 정보 집합의 크기 : {}'.format(tag_size))

단어 집합의 크기 : 70000
개체명 태깅 정보 집합의 크기 : 45


In [ ]:
X_train = src_tokenizer.texts_to_sequences(X_train)
X_test = src_tokenizer.texts_to_sequences(X_test)

In [ ]:
y_train = tar_tokenizer.texts_to_sequences(y_train)
y_test = tar_tokenizer.texts_to_sequences(y_test)

In [ ]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 256
X_train = pad_sequences(X_train, padding='post', maxlen=max_len)
X_test = pad_sequences(X_test,padding='post', maxlen = max_len )
y_train = pad_sequences(y_train, padding = 'post', maxlen=max_len)
y_test = pad_sequences(y_test, padding = 'post', maxlen =max_len)

In [ ]:
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [ ]:
word_index = src_tokenizer.word_index

In [ ]:
vocab_size = len(src_tokenizer.word_index) + 1
tag_size = len(tar_tokenizer.word_index) + 1

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))

In [ ]:
for words in word_index:
  if words in word2vec and word_index[words] < 300:
    embedding_matrix[word_index[words]] = word2vec[words]

In [ ]:
index_to_word = src_tokenizer.index_word
index_to_ner = tar_tokenizer.index_word
index_to_ner[0] = 'PAD'
from keras.callbacks import Callback
from seqeval.metrics import f1_score, classification_report

class F1score(Callback):
    def __init__(self, value = 0.0, use_char=True):
        super(F1score, self).__init__()
        self.value = value
        self.use_char = use_char

    def sequences_to_tags(self, sequences): # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
      result = []
      for sequence in sequences: # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
          tag = []
          for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다.
              pred_index = np.argmax(pred) # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
              tag.append(index_to_ner[pred_index].replace("PAD", "O")) # 'PAD'는 'O'로 변경
          result.append(tag)
      return result

    # 에포크가 끝날 때마다 실행되는 함수
    def on_epoch_end(self, epoch, logs={}):

      # char Embedding을 사용하는 경우
      if self.use_char:
        X_test = self.validation_data[0]
        X_char_test = self.validation_data[1]
        y_test = self.validation_data[2]
        y_predicted = self.model.predict([X_test, X_char_test])

      else:
        X_test = self.validation_data[0]
        y_test = self.validation_data[1]
        y_predicted = self.model.predict([X_test])

      pred_tags = self.sequences_to_tags(y_predicted)
      test_tags = self.sequences_to_tags(y_test)

      score = f1_score(pred_tags, test_tags)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(test_tags, pred_tags))

      # F1-score가 지금까지 중 가장 높은 경우
      if score > self.value:
        print('f1_score improved from %f to %f, saving model to best_model.h5'%(self.value, score))
        self.model.save('best_model.h5')
        self.value = score
      else:
        print('f1_score did not improve from %f'%(self.value))

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.models import load_model
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

In [ ]:
len(X_train)

2379

In [ ]:

model = Sequential()
model.add(Embedding(input_dim = vocab_size, 
                    output_dim = 300, 
                    input_length= max_len, 
                    weights = [embedding_matrix],
                    mask_zero = True))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(tag_size, activation="relu")))

'''
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=20, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(50, activation="relu")))
crf = CRF(tag_size)
model.add(crf)
'''






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


'\nmodel = Sequential()\nmodel.add(Embedding(input_dim=vocab_size, output_dim=20, input_length=max_len, mask_zero=True))\nmodel.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))\nmodel.add(TimeDistributed(Dense(50, activation="relu")))\ncrf = CRF(tag_size)\nmodel.add(crf)\n'

In [ ]:


#model.compile(optimizer="adam", loss="mse", metrics=["Accuracy"])
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
#model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy, f1score])

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 500, validation_data=(X_test, y_test), verbose = 1, callbacks=[F1score(use_char=False)])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
     hospital       0.46      0.19      0.27       865
      country       0.00      0.00      0.00       117
         city       0.13      0.02      0.03       261
     username       0.33      0.01      0.02        92
medicalrecord       0.00      0.00      0.00       422
       street       0.05      0.04      0.05       136
   profession       0.23      0.08      0.12       180
 organization       0.00      0.00      0.00        82
        other       0.00      0.00      0.00        13
        phone       0.00      0.00      0.00       215
        state       0.81      0.27      0.41       190
        idnum       0.54      0.04      0.07       195
          zip       1.00      0.01      0.01       140
          fax       0.00      0.00      0.00         2
       device       0.00      0.00      0.00         8
        email       0.00      0.00      0.00         1

  avg / total       0.37      0.16      0.20     22746

f1_score did not improve fr

In [ ]:
i=0 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict(np.array([X_test[i]])) # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = np.argmax(y_predicted, axis=-1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true = np.argmax(y_test[i], -1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true[true==0] = 1

print("{:25}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for w, t, pred in zip(X_test[i], true, y_predicted[0]):
    if w != 0: # PAD값은 제외함.
        print("{:25}: {:10}  {}".format(index_to_word[w], index_to_ner[t].upper(), index_to_ner[pred].upper()))

단어                       |실제값  |예측값
-----------------------------------
record                   : O           O
date:                    : O           O
2068-03-31               : B-DATE      O
eudowood                 : B-HOSPITAL  O
services                 : I-HOSPITAL  O
physician                : O           O
group                    : O           O
807                      : B-STREET    O
williams                 : I-STREET    PAD
ct                       : I-STREET    PAD
moody,                   : B-PATIENT   PAD
johnny                   : I-PATIENT   PAD
hyderabad                : B-CITY      O
,                        : O           O
wi                       : B-STATE     B-STATE
62297                    : B-ZIP       O
76324418                 : B-MEDICALRECORD  O
547-598-7353             : B-PHONE     O
03/31/2068               : B-DATE      O
history                  : O           O
of                       : O           O
present                  : O           O
illness

In [ ]:
from sklearn.metrics import confusion_matrix
all_pred = []
all_true = []
for i in range(200):
    y_pred = model.predict(np.array([X_test[i]]))
    y_pred = np.argmax(y_pred, axis=-1) 
    true = np.argmax(y_test[i], -1)
    true[true==0] = 1
    y_pred[y_pred==0] = 1

    all_pred.append(y_pred)
    all_true.append(true)

all_pred = np.array(all_pred)
all_true = np.array(all_true)



cm = confusion_matrix(all_true.reshape(51200 ),all_pred.reshape(51200 ))
print( [index_to_ner[i] for i in range(1,7)])
cm

['o', 'b-date', 'b-doctor', 'i-doctor', 'b-hospital', 'b-patient']


array([[48773,   257,     1,     0,     0,     0,     6,     2,     0,
            0,     0,     3,     0,     0,     1,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  569,    93,     1,     0,     0,     0,     2,     0,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  237,     1,    34,     6,     0,     0,     0,     1,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  143,     0,     4,    19,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  140,     1,     0,